incorrect_text → correct_text

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/raw/english_language_learning_1M.csv")

df = df[["incorrect_text", "correct_text"]].dropna()
df["input"] = "fix grammar: " + df["incorrect_text"]
df["target"] = df["correct_text"]

In [3]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
def tokenize_t5(batch):
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=64)
    targets = tokenizer(batch["target"], padding="max_length", truncation=True, max_length=64)
    inputs["labels"] = targets["input_ids"]
    return inputs

In [6]:
import pandas as pd
import torch

from datasets import Dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments
)

In [7]:
train_ds = Dataset.from_pandas(df.sample(100000))
train_ds = train_ds.map(tokenize_t5, batched=True)

args = TrainingArguments(
    output_dir="./t5_results",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    learning_rate=3e-4
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds
)

# trainer.train()

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [8]:
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

model.to(device)

Using device: mps


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [9]:
def correct_text(sentence):
    inputs = tokenizer("fix grammar: " + sentence, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    outputs = model.generate(input_ids)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
model.save_pretrained("./t5_sentence_generator")
tokenizer.save_pretrained("./t5_sentence_generator")

('./t5_sentence_generator/tokenizer_config.json',
 './t5_sentence_generator/special_tokens_map.json',
 './t5_sentence_generator/spiece.model',
 './t5_sentence_generator/added_tokens.json')